In [1]:
import control
import numpy as np
import cvxpy as cp
import scipy.optimize

In [2]:
m = 1
c = 1
k = 1

A = np.array([
    [0, 1],
    [-k/m, -c/m]
])

B = np.array([
    [0],
    [1/m]
])

C = np.array([
    [1, 0]
])

D = np.array([
    [0]
])

G = np.eye(2)
QN = (1e-1)**2*np.eye(2)  # process noise
RN = (1e-2)**2*np.eye(1)  # measurement noise
Q = np.eye(2)
R = np.eye(1)

C = np.array([[1, 0]])

L, _, _ = control.lqe(A, G, C, QN, RN)
K, _, _ = control.lqr(A, B, Q, R)

L = -L # flip sign for convention  A + LC
K = -K # flip sign for convention  A + BK

c_w = 1  # bound on processs noise 
c_v = 1  # bound on measurement noise

# check estimator and controller are stable
assert np.max(np.real(np.linalg.eig(A + L@C)[0])) < 0
assert np.max(np.real(np.linalg.eig(A + B@K)[0])) < 0

print('A', A)
print('L', L)
print('K', K)


A [[ 0.  1.]
 [-1. -1.]]
L [[-10.28516255]
 [ -2.89228433]]
K [[-0.41421356 -0.68179283]]


In [3]:
def slemma_prob(A, B, C, D, alpha0):

    # objective function
    def f(A, B, C, D, alpha):
        assert np.max(np.real(np.linalg.eig(A)[0])) < 0
        n = A.shape[0]
        m = C.shape[0]
        P = cp.Variable((n, n), 'P', PSD=True)
        mu1 = cp.Variable((1, 1), 'mu1', pos=True)
        mu2 = cp.Variable((1, 1), 'mu2', pos=True)
        constraints = [
            cp.bmat([
                [A.T@P + P@A + alpha*P, P@B],
                [B.T@P, -alpha*mu1*np.eye(n)]
            ]) << 0,
            cp.bmat([
                [C.T@C - P, C.T@D],
                [D.T@C, D.T@D - mu2*np.eye(m)]
            ]) << 0
        ]
        prob = cp.Problem(objective=cp.Minimize(mu1 + mu2), constraints=constraints)
        res = prob.solve(verbose=False)
        if prob.status == 'infeasible' or prob.status == 'infeasible_inaccurate':
            return np.inf
        elif prob.status == 'optimal' or prob.status == 'optimal_inaccurate':
            return np.sqrt(mu1.value + mu2.value)[0, 0]
        else:
            raise RuntimeError('unknown status', prob.status)

    # line search over alpha
    return scipy.optimize.minimize(
        fun=lambda x: f(A=A+L*C, B=np.eye(2), C=np.array([[1, 0]]), D=np.zeros((1, 2)), alpha=x),
        x0=alpha0,
        method='Nelder-Mead')

In [4]:
# bound on: Lv + w)
c_e = np.linalg.svd(L).S[0]*c_v + c_w
c_e

11.684094594472151

In [5]:
# C is identity here since the estimator state is the output of interest
res_est = slemma_prob(A=A+L@C, B=np.eye(2), C=np.eye(2), D=np.zeros((2, 2)), alpha0=1)
res_est


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.06712149494895603
             x: [ 7.204e+00]
           nit: 23
          nfev: 48
 final_simplex: (array([[ 7.204e+00],
                       [ 7.204e+00]]), array([ 6.712e-02,  6.721e-02]))

In [6]:
# estimator error bound
e_bound = c_e*res_est.fun
e_bound

0.784253896305987

In [7]:
# bound on: BKe + w
c_k = np.linalg.svd(B*K).S[0]*e_bound + c_w
c_k

1.6256431055356093

In [8]:
# want c matrix for x state, so we can find x bounds on MSD
res_ctrl = slemma_prob(A=A+B@K, B=np.eye(2), C=np.array([[1, 0]]), D=np.zeros((2, 1)), alpha0=1)
res_ctrl


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.14590761834438382
             x: [ 3.348e+00]
           nit: 20
          nfev: 41
 final_simplex: (array([[ 3.348e+00],
                       [ 3.349e+00]]), array([ 1.459e-01,  1.459e-01]))

In [9]:
x_bound = res_ctrl.fun*c_k
x_bound

0.23719371380666854

So the MSD will be within 24 cm of reference trajectory for all time